In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

In [13]:
# Reflect tables into SQLAlchemy ORM
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
# Set up automap_base to reflect the database components
Base = automap_base()
Base.prepare(engine, reflect=True)

In [6]:
# Let's take a look at those keys and see what the heck is going on!
Base.classes.keys()

['measurement', 'station']

In [17]:
# Assign keys to classes
Measurement = Base.classes.measurement
Station = Base.classes.station

In [8]:
# Start a session to query the database
session = Session(engine)

In [18]:
# Let's check to see if we're talking

first_row = session.query(Measurment).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f91e0fa0970>,
 'id': 1,
 'station': 'USC00519397',
 'date': '2010-01-01',
 'tobs': 65.0,
 'prcp': 0.08}

In [20]:
# Get most recent date
most_recent = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
most_recent

# Twelve months prior is 2018-08-23

('2017-08-23')

In [28]:
# Gather data for latest 12 months in database
working_data = session.query(Measurement.id, Measurement.station, Measurement.date, \
                Measurement.tobs, Measurement.prcp).\
                filter(Measurement.date > '2016-08-23').order_by(Measurement.date).all()
working_data

[(2365, 'USC00519397', '2016-08-24', 79.0, 0.08),
 (5092, 'USC00513117', '2016-08-24', 76.0, 2.15),
 (7332, 'USC00514830', '2016-08-24', 80.0, 2.28),
 (8826, 'USC00517948', '2016-08-24', 78.0, None),
 (11852, 'USC00519523', '2016-08-24', 79.0, 1.22),
 (14609, 'USC00519281', '2016-08-24', 77.0, 2.15),
 (19203, 'USC00516128', '2016-08-24', 74.0, 1.45),
 (2366, 'USC00519397', '2016-08-25', 80.0, 0.08),
 (5093, 'USC00513117', '2016-08-25', 77.0, 0.08),
 (7333, 'USC00514830', '2016-08-25', 81.0, 0.0),
 (8827, 'USC00517948', '2016-08-25', 81.0, 0.0),
 (11853, 'USC00519523', '2016-08-25', 80.0, 0.21),
 (14610, 'USC00519281', '2016-08-25', 80.0, 0.06),
 (19204, 'USC00516128', '2016-08-25', 77.0, 0.11),
 (2367, 'USC00519397', '2016-08-26', 79.0, 0.0),
 (5094, 'USC00513117', '2016-08-26', 78.0, 0.03),
 (7334, 'USC00514830', '2016-08-26', 81.0, 0.02),
 (8828, 'USC00517948', '2016-08-26', 81.0, 0.04),
 (11854, 'USC00519523', '2016-08-26', 84.0, 0.0),
 (14611, 'USC00519281', '2016-08-26', 80.0, 0.0